In [35]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # 选择任意值！我们内部会自动支持RoPE Scaling！
dtype = None # None表示自动检测。Tesla T4、V100使用Float16，Ampere+使用Bfloat16
load_in_4bit = True # 使用4比特量化以减少内存使用。可以设置为False。

# 我们支持的4比特预量化模型，下载速度快4倍 + 不会OOM（内存不足）。
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",        # Llama-3.1 15万亿token模型，速度快2倍！
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bt",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",      # 我们也上传了405B的4比特模型！
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # 新的Mistral 12b模型，速度快2倍！
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",          # Mistral v3，速度快2倍！
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",             # Phi-3.5，速度快2倍！
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",              # Gemma 2，速度快2倍！
] # 更多模型请访问 https://huggingface.co/unsloth


dtype = torch.bfloat16 # 对于Ampere+ GPU
# load_in_4bit = True # 保持不变

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-0.5B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

    # token = "hf_...", # 如果使用像meta-llama/Llama-2-7b-hf这样的门控模型，请使用此参数
)


==((====))==  Unsloth 2025.6.2: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.542 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [36]:
print('a')

a


In [37]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA 的秩 (rank)。选择任何大于0的数字！建议值有 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], # 指定要应用 LoRA 的模型层。这些通常是Attention和MLP中的线性投影层。
    lora_alpha = 16, # LoRA 的缩放因子。通常设置为与 r 相同的值。
    lora_dropout = 0, # LoRA 层的 dropout 比例。支持任何值，但设置为 0 是最优化的。
    bias = "none",    # LoRA 层的偏置类型。支持任何类型，但设置为 "none" 是最优化的。
    # [NEW] "unsloth" 模式可以减少 30% 的显存使用，并支持 2 倍大的批处理大小！
    use_gradient_checkpointing = "unsloth", # 是否使用梯度检查点以节省显存。设置为 True 或 "unsloth" 适用于非常长的上下文。
    random_state = 3407, # 用于初始化的随机种子，确保结果可复现。
    use_rslora = False,   # 是否使用 Rank Stabilized LoRA (rSLORA)。我们支持这个特性。
    loftq_config = None, # LoftQ 配置。我们支持 LoftQ 量化。
)

In [38]:
# ----------------- 1. 定义数据格式化功能 -----------------
from datasets import load_dataset

# Alpaca 格式的提示模板。这个模板定义了如何组织指令、输入和响应来构建模型的训练数据。
# The alpaca_prompt template defines how to combine data into a full prompt.
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 模型的结束标记。在训练时，必须将此标记添加到每个训练样本的末尾，否则模型可能会无限生成文本。
# The End-Of-Sequence token must be added to the end of each training sample.
EOS_TOKEN = tokenizer.eos_token # 必须添加 EOS_TOKEN (End Of Sentence Token)

# 这是一个用于格式化数据集样本的函数。
# This is a function to format the dataset samples.
# 找到第5个单元格中的这个函数
def formatting_prompts_func(examples):
    # 原来的代码只使用了两列
    # instructions = examples["INSTRUCTION"]
    # outputs      = examples["RESPONSE"]
    
    # 修改后：同时获取 summary, think, content三列
    instructions = examples["instruction"]
    inputs       = [""] * len(instructions) # 输入部分通常留空
    thoughts     = examples["thought"]        # 获取思考过程
    contents     = examples["output"]      # 获取最终的笑话内容
    
    texts = []
    # 同时遍历指令、输入、思考和内容
    for instruction, input_text, thought, content in zip(instructions, inputs, thoughts, contents):
        # 核心：将思考过程和最终回答组合成一个完整的输出
        # 我们用明确的标签"思考："和"回答："来引导模型学习这种格式
        output = f"思考：{thought}\n\n回答：{content}"
        
        # 使用Alpaca模板填充，并将组合后的output填入Response部分
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
        
    return { "text" : texts, }

# ... 对 train_dataset 和 eval_dataset 的 .map(formatting_prompts_func, ...) 调用保持不变

# ----------------- 2. 加载并准备数据集 -----------------

# 1. 加载完整的数据集

full_dataset = load_dataset("LooksJuicy/ruozhiba-punchline", split="train")

# 2. 将其切分为训练集和验证集 (95% 训练, 5% 验证)
# Split it into a training and an evaluation set (95% train, 5% eval)
# test_size=0.05 means 5% of the data will be used for the test set.
# seed=42 ensures that the split is the same every time, for reproducibility.
print("正在切分数据集...")
split_dataset = full_dataset.train_test_split(test_size=0.05, seed=42)

# 3. 分别获取训练集和验证集
# Get the training and evaluation sets respectively.
train_dataset = split_dataset['train']
# .train_test_split names the split part 'test' by default.
eval_dataset = split_dataset['test'] 

print(f"\n数据集切分完成！")
print(f"训练集大小: {len(train_dataset)}")
print(f"验证集大小: {len(eval_dataset)}")

# 打印列名以确认
print("训练集的列名是:", train_dataset.column_names)

# ----------------- 3. 格式化数据集 -----------------

print("\n正在格式化训练集...")
train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched = True,
)

print("正在格式化验证集...")
eval_dataset = eval_dataset.map(
    formatting_prompts_func,
    batched = True,
)

print("\n数据处理全部完成！")






正在切分数据集...

数据集切分完成！
训练集大小: 3267
验证集大小: 172
训练集的列名是: ['raw', 'instruction', 'thought', 'output']

正在格式化训练集...
正在格式化验证集...

数据处理全部完成！


In [39]:
import os
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 设置W&B环境变量以修复错误并更好地组织运行
os.environ["WANDB_PROJECT"] = "Qwen2-Finetune"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["WANDB_NOTEBOOK_NAME"] = "Qwen2.5-weitiaoban.ipynb"

# 初始化 SFTTrainer 进行监督微调 (Supervised Fine-Tuning)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    # --- 关键修改 1: 增加数据处理的进程数 ---
    dataset_num_proc = 8,  # [!code ++] 将数据预处理的CPU进程数从2增加到8 (您可以根据您CPU的核心数调整)
    packing = True,
    args = TrainingArguments(
        # --- 关键修改 2: 开启内存锁定 ---
        dataloader_pin_memory = True, # [!code ++] 加速数据从CPU到GPU的传输
        
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        #max_steps = 1000,
        num_train_epochs = 10, # 如果设置了max_steps，此参数会被覆盖。注释掉以备用。
        eval_strategy = "steps",
        eval_steps = 100,
        save_steps = 200,
        load_best_model_at_end = True,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1234,
        output_dir = "outputs",
        report_to = "wandb",
        run_name = "qwen2.5-optimized", # 更新一下运行名称
    ),
)


In [40]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.542 GB.
1.252 GB of memory reserved.


In [41]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,267 | Num Epochs = 10 | Total steps = 510
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 8,798,208/500,000,000 (1.76% trained)


Step,Training Loss,Validation Loss
100,1.473700,1.481495
200,1.293900,1.410602
300,1.164600,1.417495
400,1.045300,1.452323
500,0.974600,1.489789


wandb: Adding directory to artifact (./outputs/checkpoint-200)... Done. 0.2s
wandb: Adding directory to artifact (./outputs/checkpoint-400)... Done. 0.2s
wandb: Adding directory to artifact (./outputs/checkpoint-510)... Done. 0.2s


In [42]:
# @title 显示最终内存和时间统计
# 计算模型训练期间峰值预留显存（以GB为单位）
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# 计算仅用于LoRA训练的峰值预留显存（从总峰值中减去训练开始前的显存）
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
# 计算峰值预留显存占总显存的百分比
used_percentage = round(used_memory / max_memory * 100, 3)
# 计算仅用于LoRA训练的峰值预留显存占总显存的百分比
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

# 打印训练运行时长（秒）
print(f"{trainer_stats.metrics['train_runtime']} 秒用于训练。")
# 打印训练运行时长（分钟）
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} 分钟用于训练。"
)
# 打印峰值预留显存量（GB）
print(f"峰值预留显存 = {used_memory} GB。")
# 打印仅用于训练的峰值预留显存量（GB）
print(f"用于训练的峰值预留显存 = {used_memory_for_lora} GB。")
# 打印峰值预留显存占最大可用显存的百分比
print(f"峰值预留显存占最大显存的百分比 = {used_percentage} %。")
# 打印仅用于训练的峰值预留显存占最大可用显存的百分比
print(f"用于训练的峰值预留显存占最大显存的百分比 = {lora_percentage} %。")


430.5363 秒用于训练。
7.18 分钟用于训练。
峰值预留显存 = 1.252 GB。
用于训练的峰值预留显存 = 0.0 GB。
峰值预留显存占最大显存的百分比 = 5.318 %。
用于训练的峰值预留显存占最大显存的百分比 = 0.0 %。


In [45]:
# alpaca_prompt = Copied from above # alpaca_prompt 变量从之前的代码中复制而来 (这里假设它已经定义)

# 启用模型以进行原生2倍更快的推理。这会优化模型以进行生成，可能禁用一些训练相关的特性。
FastLanguageModel.for_inference(model)

# 使用 tokenizer 处理输入。
inputs = tokenizer(
[
    # 使用之前定义的 alpaca_prompt 模板来格式化输入。
    alpaca_prompt.format(
        "望远镜可以看见地球吗", # instruction: 任务指令
        "", # input: 提供给模型的输入或上下文，这里是斐波那契数列的前几项。
        "", # output: 生成时此字段留空。模型会根据 instruction 和 input 来生成 output。
    )
],
return_tensors = "pt" # 指定返回 PyTorch 张量。
).to("cuda") # 将输入张量移动到 CUDA 设备（GPU）上。

# 使用模型生成文本。
outputs = model.generate(
    **inputs, # 将处理后的输入张量作为关键字参数传递给 generate 方法。
    max_new_tokens = 128, # 限制模型生成的新 token 的最大数量为 64。
    use_cache = True # 启用 KV 缓存，可以加速重复生成（例如在逐个 token 生成时）。
)

# 使用 tokenizer 将生成的 token ID 列表解码回人类可读的文本。
tokenizer.batch_decode(outputs)


['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n望远镜可以看见地球吗\n\n### Input:\n\n\n### Response:\n思考：可以通过夸张和讽刺的手法来制造笑点。望远镜是用来看远处的，而地球是地球，望远镜只能看到地球的表面，所以望远镜只能看见地球的表面。这里将望远镜的这种局限性夸张化，形象地描绘了望远镜的局限性。\n\n回答：望远镜只能看见地球的表面<|endoftext|>']

In [46]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [51]:
# 这个 `if False:` 块意味着其中的代码当前不会被执行。
# 但它展示了如何加载一个已经微调过的模型（例如，保存为 "lora_model"）以进行推理。
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # 你用于训练的模型名称，这里是示例模型路径
        max_seq_length = max_seq_length, # 最大序列长度，与训练时保持一致
        dtype = dtype, # 数据类型，与训练时保持一致（例如：torch.bfloat16 或 torch.float16）
        load_in_4bit = load_in_4bit, # 是否以4比特量化加载模型，与训练时保持一致
    )
    # 启用模型以进行原生2倍更快的推理。这会优化模型以进行生成，可能禁用一些训练相关的特性。
    FastLanguageModel.for_inference(model)

# alpaca_prompt = You MUST copy from above! # 这里假设 `alpaca_prompt` 变量已在之前的代码中定义并复制到此处。

# 使用 tokenizer 处理输入。
inputs = tokenizer(
[
    # 使用之前定义的 `alpaca_prompt` 模板来格式化输入。
    alpaca_prompt.format(
        "装6000的电脑要花多少钱", # instruction: 任务指令
        "", # input: 提供给模型的输入或上下文，这里留空。
        "", # output: 生成时此字段必须留空。模型会根据 instruction 和 input 来生成 output。
    )
],
return_tensors = "pt" # 指定返回 PyTorch 张量。
).to("cuda") # 将输入张量移动到 CUDA 设备（GPU）上。

from transformers import TextStreamer
# 初始化 TextStreamer。它会将 tokenizer 用于解码生成的 token 并实时打印到控制台。
text_streamer = TextStreamer(tokenizer)

# 使用模型生成文本。
# **inputs 解包输入张量。
# streamer = text_streamer 将 TextStreamer 对象传递给 generate 方法，使得生成过程中的 token 可以实时流式输出。
# max_new_tokens = 128 限制模型生成的新 token 的最大数量为 128。
# `_ = ...` 表示我们不关心 `generate` 方法的返回值（因为输出会通过 `streamer` 打印）。
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
装6000的电脑要花多少钱

### Input:


### Response:
思考：可以通过夸张和讽刺的手法来制造笑点。通常我们都知道装6000的电脑是需要花费大量的钱，但是这里却用了一个非常夸张的表述，即“装6000的电脑，需要100000000000000000000000000000000000000000000000000000000000000000000000000000


In [16]:
if False: # 这个 'if False:' 块表示其中的代码当前不会被执行。
          # 它是作为一种替代方案或演示，强烈不建议使用此方法，如果可能的话请优先使用 Unsloth 的方法。
    from peft import AutoPeftModelForCausalLM # 从 peft 库导入 AutoPeftModelForCausalLM，用于自动加载 LoRA 微调过的因果语言模型。
    from transformers import AutoTokenizer # 从 transformers 库导入 AutoTokenizer，用于自动加载与模型对应的分词器。

    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # 你用于训练的模型路径（例如，保存 LoRA 适配器的目录）。
        load_in_4bit = load_in_4bit, # 以4比特量化方式加载模型，与训练时保持一致以节省显存。
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model") # 从相同的路径加载与模型对应的分词器。


In [17]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [18]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )